<a href="https://colab.research.google.com/github/patecm/rapidEELS/blob/main/EELS_model_training_GH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%% === USER PREFERENCES
# --- Specify location of EELS files
path_files = '/tmp/eels/ex situ SFO 2-5-20'
path_dark = '/tmp/eels/bkg_sub.dm4'

# -- General Imports
import numpy as np
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import os
from os import listdir, mkdir
from os.path import isfile, join, exists
from datetime import datetime, date
from pytz import timezone
#import progressbar
import sys
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, RobustScaler
from skimage.measure import block_reduce

# -- Import previously installed packaged
# LMFit
from lmfit.models import GaussianModel, PowerLawModel
# Hyperspy - for reading dm4 files
# Ignore errors about the GUI modules not being installed. We don't use the GUI for this anyway    
import hyperspy.api as hs

# -- Import Tensorflow Packages
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers, losses, Input
from tensorflow.keras import backend as K

from tensorflow.keras.models import save_model, Model
from tensorflow.keras.layers import Dropout, Input, Reshape, Activation
from tensorflow.keras.layers import BatchNormalization, Dense, Flatten
from tensorflow.keras.layers import Conv1D, Conv1DTranspose

from tensorflow.keras.losses import mse, sparse_categorical_crossentropy
from tensorflow.keras.losses import binary_crossentropy, categorical_crossentropy
from tensorflow.keras.metrics import BinaryCrossentropy

from tensorflow.keras.regularizers import l1, l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint 

# Set random seeds
tf.random.set_seed(42)
np.random.seed(42)

# Double check tensorflow install and python version
print('Tensorflow version')
print(tf.__version__)
print(tf.test.is_built_with_cuda())

print("Python version")
print (sys.version)


#%% === FUNCTIONS
def readDM42(fname, emin=440., emax=580., path_files=path_files, path_dark=path_dark):
    # Input: fname - filename of the dm4 file to load
    #        emin - minimum energy for croppig spectrum
    #        emin - maximum energy for croppig spectrum
    #        path_files - location of dm4 file to load
    #        path_dark - full path (inld. filename) to the dark file

    # Output: hs_pixels - 400FPS pixels, 560 ebins (440 to 580eV)
    #        y_groundtruth - 1FPS ground truth, 560 bin (440 to 580eV)

    # --- Load dark file
    dark = hs.load(path_dark)
    # --- Load DM4 with HyperSpy
    hs_template = hs.load(os.path.join(path_files, fname))
    hs_pixels =  hs_template + dark
    # Rebin the spectrum from 0.125eV to 0.25eV with Hyperspy
    hs_pixels = hs_pixels.rebin(scale=[1,1,2], crop=True, out=None)
    # crop from 440 to 580 eV
    hs_pixels = hs_pixels.isig[emin:emax]
    #print(hs_pixels.data.shape)
    
  
    # --- Create Ground Truths @ 1FPS (440 to 580eV)
    y_1fps = []
    for j in range(16):
        section_min = 40*j
        section_max = 40*(j+1)
        placeholder = hs_pixels.copy().inav[section_min:section_max, 0:10]
        placeholder = placeholder.mean(axis=0).mean(axis=0)

        # Append to y_train
        y_1fps.append(placeholder.data)
    y_1fps = np.asarray(y_1fps)
    ebins = y_1fps.shape[-1]

    # Repeat each spectrum to match w/ corresponding area of pixels in SI
    y_groundtruth = []
    for s in y_1fps:
        spec = np.repeat([s], [400], axis=0).tolist()
        y_groundtruth.append(spec)
    y_groundtruth = np.asarray(y_groundtruth)

    #hs_pixels = hs_pixels.isig[520.:580.]
    hs_pixels = np.asarray(hs_pixels.data)
    xmax, ymax = hs_pixels.shape[0], hs_pixels.shape[1]
    # crop end of ground truth SI with no corresponding pixels
    y_groundtruth = y_groundtruth.reshape(xmax, -1, y_groundtruth.shape[-1])
    y_groundtruth = y_groundtruth[:, 0:ymax, :]
    
    # Return:
    #y_train: 400FPS, 440 to 580eV
    #y_groundtruth: 1FPS 440 to 580eV
    return(hs_pixels, y_groundtruth)

def fitgaussian(spectra, axis):
    mod = GaussianModel()
    prepeak_pars = mod.guess(spectra, x=axis)
    prepeak_out = mod.fit(spectra, prepeak_pars, x=axis)
    return(prepeak_out)

def readDM4(fname, emin=520., emax=580., path_files=path_files, path_dark=path_dark):
    # Input: fname - filename of the dm4 file to load
    #        emin - minimum energy for croppig spectrum
    #        emin - maximum energy for croppig spectrum
    #        path_files - location of dm4 file to load
    #        path_dark - full path (inld. filename) to the dark file

    # Output: hs_template - 
    #        sample_nobackground.data - 
    #        y_sample - 

    # --- Load dark file
    dark = hs.load(path_dark)
    # --- Load DM4 with HyperSpy
    hs_template = hs.load(os.path.join(path_files, fname))
    hs_template =  hs_template + dark
    # Rebin the spectrum from 0.125eV to 0.25eV with Hyperspy
    hs_template = hs_template.rebin(scale=[1,1,2], crop=True, out=None)

    y_train = []
    y_sample = []
    # --- Create Ground Truths @ 1FPS
    # PowerLaw Denoise the 10x40 1FPS sections 
    for j in range(16):
        section_min = 40*j
        section_max = 40*(j+1)
        placeholder = hs_template.copy().inav[section_min:section_max, 0:10]
        placeholder = placeholder.mean(axis=0).mean(axis=0).remove_background(signal_range=(500.,524.),
                          background_type='PowerLaw',
                          fast=False,
                          return_model=False)

        # Crop to Oxygen Peaks range using user specified emin and emax
        placeholder = placeholder.isig[emin:emax]
        # Append to y_train
        y_train.append(placeholder.data)
    y_train = np.asarray(y_train)

    # PowerLaw Denoise the mean of the entire SI
    # (Used for making some of the graphics)
    si_nobackground = hs_template.copy().sum(axis=0).sum(axis=0).remove_background(signal_range=(500.,524.),
                        background_type='PowerLaw',
                        fast=False,
                        return_model=False)

    # Repeat each spectrum to match w/ corresponding area of pixels in SI
    for s in y_train:
        spec = np.repeat([s], [400], axis=0).tolist()
        y_sample.append(spec)
    y_sample = np.asarray(y_sample).reshape(-1, 240)
    y_sample = np.asarray(y_sample).reshape(10, -1, 240)

    # Crop the last 38 that are < 1FPS
    y_sample = y_sample.reshape(10, -1, y_train.shape[-1])
    y_sample = y_sample[:, 0:638, :]

    # keep only portion of SI UPTO the Oxygen peak
    # This is needed to correctly denoise later
    hs_template = hs_template.isig[440.:emax] 
    #hs_template = hs_template.inav[0:638, :]

    si_nobackground = si_nobackground.isig[emin:emax]

    return(hs_template, si_nobackground.data, y_sample)

def plot_losses(hist):
    # Plot of loss for trained model
    plt.title('Model loss')

    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])

    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Val'], loc='upper right')
    plt.show()

def expand_yvalues(ydata, xmax, ymax):
    y_classes = []
    for y in ydata:
        y_classes.extend([y] * (xmax*ymax))
    y_classes = np.asarray(y_classes)
    return(y_classes)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Tensorflow version
2.4.1
True
Python version
3.7.10 (default, Feb 20 2021, 21:17:23) 
[GCC 7.5.0]


In [ ]:
#%% === LOAD TRAINING ===
# Note: The new version of Hyperspy gives an error about Axis calibration mismatch
# This can be safely ignored for now until the code can be updated to not throw the error message
# --- Read DM4 Names ---
# get filenames
exsitu_filenames = [f for f in listdir(path_files) if isfile(join(path_files, f))]

# Specify which samples to keep for the test data set
exsitu_filenames_test = ['EELS thin 1_1.dm4', 'EELS thick 1_1.dm4',
                         'EELS thin 1_1a.dm4', 'EELS thick 1_1a.dm4']

# Get names of the training data set
exsitu_filenames_train = [f for f in exsitu_filenames if f not in exsitu_filenames_test]

# Print training filenames for referencing later in graphs
for i in np.arange(len(exsitu_filenames_train)/2):
    print('{} - {}     {} - {}'.format(int(i), exsitu_filenames_train[int(i)],
                                       int(i+10),
                                       exsitu_filenames_train[int(i)+10]))
# Read dm4 files
X_train = []
y_train = []
start = time.process_time()
for fname in exsitu_filenames_train:
    print('loading: {}'.format(fname))
    sample, sample_groundtruth = readDM42(fname, emin=440., emax=580., path_files=path_files, path_dark=path_dark)

    X_train.append(sample) #raw 400FPS spectral data, 560 bins long 440-580eV
    y_train.append(sample_groundtruth) #raw 1FPS Ground Truth 560 bins long 440-580eV


# Process/trim data after loaded
y_train = np.asarray(y_train) #raw 1FPS Ground Truth 560 bins long 440-580eV
X_train = np.asarray(X_train)[:, :, :, -240:] #400FPS spectral data, 240 bins long 520-580eV
xmax, ymax = X_train.shape[1], X_train.shape[2]

print(X_train.shape)
print(y_train.shape)

end = time.process_time() - start
print('Runtime is {} sec to load {} files'.format(end, X_train.shape[0]))

# BACKGROUND SUBTRACT Y_TRAIN
# eV range for background subtraction
eV_increment = 0.25
emin = 440
axes = np.arange(0,np.shape(y_train)[-1])*0.25+emin

index_min = int((498 - emin) / eV_increment)
index_max = int((528 - emin) / eV_increment)
x = axes[index_min:index_max]

# Fit powerlaw
start = time.process_time()

y_train = y_train.reshape(-1, y_train.shape[-1])
y_train_nobg = []
counter = 0
for counter, s in enumerate(y_train):
    # Fit Powerlaw model to before pre-peak
    fit_region = s.copy()[index_min:index_max]

    # Model
    mod = PowerLawModel()
    pars = mod.guess(fit_region, x=x)
    out = mod.fit(fit_region, pars, x=x)
    #Powerlaw subtract y=A*x*exp(-r)
    A = out.best_values['amplitude']
    r = out.best_values['exponent']

    # Apply fit to sample
    fx = A * np.power(axes, r)
    y_train_nobg.append(s - fx)
    if counter % 10000 == 0:
        print('...Finished fitting {} out of {} spectra'.format(counter, y_train.shape[0]))
        
y_train = np.array(y_train_nobg.copy()) #convert from list to arrayx_max = int((528 - emin) / eV_increment)


# eV range for y_test (520 to 580eV)
index_min = int((520 - emin) / eV_increment)
index_max = int((580 - emin) / eV_increment)
y_train = y_train[:, index_min:index_max]
y_train = y_train.reshape(len(exsitu_filenames_train), -1, y_train.shape[-1])

end = time.process_time()
print('Runtime is {} for background subtracat on batch of shape {}'.format(end-start, y_train.shape))

# Reshape
X_train = X_train.reshape(-1, np.shape(X_train)[-1])
y_train = y_train.reshape(-1, np.shape(y_train)[-1])

# Robust Scaling
robust_scaler = RobustScaler(quantile_range=(0.25, 0.75)).fit(X_train.copy())
X_train = robust_scaler.fit_transform(X_train)
X_train_scaled = X_train.copy()

# Random Noise (Option 1)
noise_factor = 0.4
X_train = X_train.copy() + noise_factor * tf.random.normal(shape=X_train.shape) 
X_train = X_train + noise_factor * np.random.normal(size=X_train.shape)

# Poisson Noise (Option 2)
#noisy_samples = []
#for i, sample in enumerate(X_train):
#  noise_mask = np.random.poisson(sample)
#  noisy_samples.append(sample + noise_mask)
#X_train = np.array(noisy_samples)

# Make 3D for correct input to NNs
X_train = np.atleast_3d(X_train).astype('float32')
y_train = np.atleast_3d(y_train).astype('float32')

print('Making binary and quad classes...')
# --- Make Binary Classes - Green v. Annealed
filename_binary_train = np.where(~np.char.endswith(exsitu_filenames_train, 'a.dm4'),
                           exsitu_filenames_train, 1)
filename_binary_train = np.where(~np.char.endswith(filename_binary_train, '.dm4'),
                           filename_binary_train, 0)
# Expand the [0,1] values from sample name to each pixel - 400FPS
y_binary_train = expand_yvalues(filename_binary_train, xmax=xmax, ymax=ymax)
y_binary_train = y_binary_train.astype('int32')

# Encode classes for training in Keras
# Binary class
y_binary_train = np.atleast_3d(tf.keras.utils.to_categorical(y_binary_train, num_classes=2))
print(y_binary_train.shape)
print(X_train.shape[0]*X_train.shape[1])

In [ ]:
#%% === TRAIN AUTOENCODER ===
def create_CNN(X_train, latent_dim):

    # --- MODEL PARAMETERS
    latent_dims = 5
    batch_size = 512 
    epochs = 500 

    dropout = 0.2
    l1_norm = 3e-4
    learning_rate = 1e-4
    #loss = 'mse'

    # Encoder/Decoder layers and filters per layer (v2)
    aec_layer_filters = [8, 16, 16, 32, 64]
    aec_layer_kernels = [7, 7, 5, 5, 3]

    spectrum_size = X_train.shape[1]
    input_shape = (spectrum_size, 1)

    # BUILD CNN AEC
    encoder_input = Input(shape=input_shape, name='encoder_input')
    x = encoder_input
    for i, lf in enumerate(aec_layer_filters):
        x = Conv1D(filters=lf, kernel_size=aec_layer_kernels[i],
                   strides=2,
                   padding='same',
                   activation='relu',
                   name='conv1D{}'.format(i))(x)  
        if i < len(aec_layer_filters)-1:
            x = Dropout(dropout)(x)

    # Shape info needed to build Decoder Model
    shape = K.int_shape(x)

    # --- Latent Space
    x = Flatten(name='flattened')(x)
    latent_space = Dense(latent_dim, name='latent_space', activation='relu')(x)

    # Reshape input to decoder after Dense layer
    x = Dense(shape[1] * shape[2])(latent_space)
    x = Reshape((shape[1], shape[2]))(x)     

    # --- Decoder
    x = Conv1DTranspose(filters=aec_layer_filters[-1],
                        kernel_size=aec_layer_kernels[-1],
                        strides=2,
                        padding='same',
                        output_padding=1,
                        activation='relu')(x)
                
    for i, lf in enumerate(aec_layer_filters[0:-1][::-1]):
        x = Conv1DTranspose(filters=lf, kernel_size=aec_layer_kernels[0:-1][::-1][i],
                            strides=2,
                            padding='same',
                            activation='relu')(x)

    # Spectrum
    decoded = Conv1DTranspose(filters=1, kernel_size=aec_layer_kernels[0], padding='same',
                              activation='linear',
                              name='decoded_spectrum')(x) 
    autoencoder = Model(encoder_input, decoded)

    # Compile model
    autoencoder.compile(loss=tf.keras.losses.MeanSquaredError(),
                        optimizer='adam')
    return(autoencoder)

# Create Model
aec_model = create_CNN(X_train, latent_dims)
aec_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, 240, 1)]          0         
_________________________________________________________________
conv1D0 (Conv1D)             (None, 120, 8)            48        
_________________________________________________________________
dropout (Dropout)            (None, 120, 8)            0         
_________________________________________________________________
conv1D1 (Conv1D)             (None, 60, 16)            656       
_________________________________________________________________
dropout_1 (Dropout)          (None, 60, 16)            0         
_________________________________________________________________
conv1D2 (Conv1D)             (None, 30, 32)            2592      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30, 32)            0     

In [ ]:
#%% === TRAINING PARAMETERS ===
# Autoencoder
aec_batch_size =  512
aec_epochs = 500
# Classifier
cf_batch_size =  512
cf_epochs = 500


#%% === TRAIN DENOISING AEC ===
start_time = datetime.now().strftime("%H:%M:%S")
print("Start training at Time = {}".format(start_time))
hist = aec_model.fit(X_train, y_train,
                     epochs=aec_epochs,
                     batch_size=aec_batch_size,
                     verbose=1,
                     shuffle=True)

# --- Save AEC model - Optional ---
#aec_path = '/content/drive/MyDrive/Final EELS/Other March Models'
#aec_filename = 'aecModel-e{}-bs{}-ld{}-{}-v3'.format(epochs, batch_size, latent_dims, date.today().strftime("%b%d"))
#aec_name = join(aec_path, aec_filename)
#aec_model.save(aec_name)
#print('Model saved to: {}'.format(aec_name))

end_time = datetime.now().strftime("%H:%M:%S")
print("Started training at: {} Ended at: {}".format(start_time, end_time))

#%% === TRAIN CLASSIFIER ===
# --- Separate Encoder from AEC Model ---
encoder_model = Model(inputs=aec_model.input,
                      outputs=aec_model.get_layer('latent_space').output)

encoder_model.trainable = False  # Freeze the inner model
assert encoder_model.trainable == False  # All layers in model are now frozen
assert encoder_model.layers[0].trainable == False  #trainable is propagated recursively
encoder_model.compile()

# --- Create Classifier
classifier_input = Input(shape=(X_train.shape[1], 1), name='classifier_input')
x = encoder_model(classifier_input, training=False)

classifier_output = Dense(2, activation='softmax')(x) 
classifier_model = Model(classifier_input, classifier_output)
classifier_model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                         optimizer='adam',
                         metrics=['accuracy', 'categorical_accuracy'])
#classifier_model.summary()

# --- Train Classifier Model 
start = time.perf_counter()
hist_classifier = classifier_model.fit(X_train, y_binary_train,
                                       epochs=cf_epochs,
                                       batch_size=cf_batch_size,
                                       shuffle=True)

# --- Save model (must have GDrive authorized) - Optional
#classifier_path = 'tmp/models/'
#classifier_filename = 'classifierModel-e{}-bs{}-ld{}-{}-quadclass-v3'.format(epochs, batch_size, latent_dims, date.today().strftime("%b%d"))
#classifier_name = join(classifier_path, classifier_filename)
#classifier_model.save(classifier_name)
#print('Model saved to: {}'.format(classifier_name))

end = (time.perf_counter() - start)
end_time = datetime.now().strftime("%H:%M:%S")
print('Classifier Total training time: {} sec'.format(end))
print("Started training models at: {} Ended at: {}".format(start_time, end_time))